## This notebook will be mainly used for the capstone project.

In [97]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [98]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Segmenting and Clustering Neighborhoods in Toronto

In [99]:
#extract table from wikipedia

from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = read_html(page,attrs={"class":"wikitable"})


In [100]:
df = table[0]

In [101]:
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [102]:
#drop rows where borough is not assigned

df = df[df.Borough!= "Not assigned"]
df = df.reset_index(drop=True)

In [103]:
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [104]:
df = df.groupby(['Postcode', 'Borough'], sort = False).agg(','.join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [105]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [106]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [107]:
!wget -q -O 'Toronto_long_lat.csv'  http://cocl.us/Geospatial_data
dft = pd.read_csv('Toronto_long_lat.csv')
dft.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Import required libraries

In [108]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors
 
from sklearn.cluster import KMeans
import folium

### Merge dft and df on postal codes

In [109]:
#rename postal code to postcode

dft.columns = ['Postcode', 'Latitude', 'Longitude']

dft = pd.merge(df, dft, on = 'Postcode')

dft.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [110]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Making a folium Map of Toronto with markers for each neighborhood.

In [111]:
toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

for latitude, longitude, borough, Neighbourhood in zip(dft['Latitude'], dft['Longitude'], dft['Borough'], dft['Neighbourhood']):
    label = "{}, {}".format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.5,
        parse_html = False).add_to(toronto_map)
    
toronto_map

K-means clustering

In [112]:
k = 4
dftemp = dft
dftemp = dftemp.drop(['Neighbourhood','Borough','Postcode'], axis = 1)


In [113]:
dftemp

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494
5,43.667856,-79.532242
6,43.806686,-79.194353
7,43.745906,-79.352188
8,43.706397,-79.309937
9,43.657162,-79.378937


In [114]:
kmeans = KMeans(n_clusters = k, random_state = 0).fit(dftemp)

kmeans.labels_[1:10]

array([3, 0, 2, 0, 1, 3, 2, 0, 0], dtype=int32)

In [115]:
cluster_labels = pd.DataFrame(kmeans.labels_)[0]
cluster_labels.head()

0    3
1    3
2    0
3    2
4    0
Name: 0, dtype: int32

In [116]:
dft['Cluster'] = cluster_labels

In [117]:
dft.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,3
1,M4A,North York,Victoria Village,43.725882,-79.315572,3
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,2
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0


## Creating a map with clusters

In [119]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 10)

x = np.arange(k)
ys = [i + x + (i*x) ** 2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

marker_color = []
for latitude, longitude, poi, cluster in zip(dft["Latitude"], dft['Longitude'], dft['Neighbourhood'], dft['Cluster']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
    
map_clusters